In [39]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [40]:
data_csv = pd.read_parquet("train.parquet")

In [41]:
data_csv.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,log_bike_count
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.000000
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.693147
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.000000
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,1.609438
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,2.302585


In [42]:
missing_values = data_csv.isnull().sum()
data_types = data_csv.dtypes
summary_statistics = data_csv.describe()

missing_values, data_types, summary_statistics

(counter_id                   0
 counter_name                 0
 site_id                      0
 site_name                    0
 bike_count                   0
 date                         0
 counter_installation_date    0
 counter_technical_id         0
 latitude                     0
 longitude                    0
 log_bike_count               0
 dtype: int64,
 counter_id                         category
 counter_name                       category
 site_id                               int64
 site_name                          category
 bike_count                          float64
 date                         datetime64[ns]
 counter_installation_date    datetime64[ns]
 counter_technical_id               category
 latitude                            float64
 longitude                           float64
 log_bike_count                      float64
 dtype: object,
             site_id     bike_count                           date  \
 count  4.551630e+05  455163.000000                 

In [43]:
external_data = pd.read_csv("external_data.csv")

In [44]:
external_data.head()

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,hnuage1,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4
0,7149,2021-01-01 00:00:00,100810,80,1,270,1.8,272.75,272.15,96,...,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7149,2021-01-01 03:00:00,100920,110,3,300,1.7,271.25,270.95,98,...,1500.0,2.0,3.0,3000.0,NaN,NaN,NaN,NaN,NaN,NaN
2,7149,2021-01-01 06:00:00,100950,30,3,290,2.6,271.95,271.65,98,...,480.0,4.0,6.0,2000.0,6.0,3.0,3000.0,NaN,NaN,NaN
3,7149,2021-01-01 09:00:00,101100,150,2,280,1.7,272.45,272.05,97,...,1740.0,3.0,3.0,2800.0,NaN,NaN,NaN,NaN,NaN,NaN
4,7149,2021-01-01 12:00:00,101110,30,0,50,1.0,276.95,274.15,82,...,330.0,4.0,6.0,570.0,7.0,6.0,810.0,NaN,NaN,NaN


In [45]:
from datetime import datetime

# Convert the 'date' columns in both datasets to datetime for alignment
data_csv['date'] = pd.to_datetime(data_csv['date'])
external_data['date'] = pd.to_datetime(external_data['date'])

# Interpolating and forward filling the external data
# Selecting continuous and categorical columns
continuous_cols = external_data.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = external_data.select_dtypes(exclude=['float64', 'int64']).columns

external_data_interpolated = external_data.copy()
external_data_interpolated[continuous_cols] = external_data_interpolated[continuous_cols].interpolate()

external_data_interpolated[categorical_cols] = external_data_interpolated[categorical_cols].ffill()

# Merging the datasets without setting 'date' as the index
merged_data = pd.merge_asof(data_csv.sort_values('date'), 
                                     external_data_interpolated.sort_values('date'),
                                     on='date', 
                                     direction='nearest')

In [46]:
merged_data.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,...,hnuage1,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4
0,100056332-104056332,Pont de Bercy SO-NE,100056332,Pont de Bercy,0.0,2020-09-01 01:00:00,2019-12-11,Y2H19070378,48.83848,2.37587,...,2750.0,1.0,2.4,5300.0,6.2,3.333333,2704.0,5.602317,0.0,5969.88417
1,100047547-104047547,6 rue Julia Bartet NE-SO,100047547,6 rue Julia Bartet,4.0,2020-09-01 01:00:00,2018-11-28,Y2H18086323,48.82636,2.30303,...,2750.0,1.0,2.4,5300.0,6.2,3.333333,2704.0,5.602317,0.0,5969.88417
2,100047547-103047547,6 rue Julia Bartet SO-NE,100047547,6 rue Julia Bartet,2.0,2020-09-01 01:00:00,2018-11-28,Y2H18086323,48.82636,2.30303,...,2750.0,1.0,2.4,5300.0,6.2,3.333333,2704.0,5.602317,0.0,5969.88417
3,100057380-103057380,Totem Cours la Reine O-E,100057380,Totem Cours la Reine,0.0,2020-09-01 01:00:00,2020-02-11,YTH19111509,48.86462,2.31444,...,2750.0,1.0,2.4,5300.0,6.2,3.333333,2704.0,5.602317,0.0,5969.88417
4,100047548-103047548,Face au 25 quai de l'Oise NE-SO,100047548,Face au 25 quai de l'Oise,2.0,2020-09-01 01:00:00,2018-11-28,Y2H18086324,48.89141,2.38482,...,2750.0,1.0,2.4,5300.0,6.2,3.333333,2704.0,5.602317,0.0,5969.88417


In [48]:
# Convert 'date' to datetime for feature extraction
merged_data['date'] = pd.to_datetime(merged_data['date'])

# Re-add date-related features
merged_data['hour'] = merged_data['date'].dt.hour
merged_data['day_of_week'] = merged_data['date'].dt.dayofweek
merged_data['month'] = merged_data['date'].dt.month
merged_data['is_weekend'] = merged_data['day_of_week'].isin([5, 6]).astype(int)  # 0 for weekdays, 1 for weekends
merged_data['is_night'] = merged_data['hour'].apply(lambda x: 1 if (x < 6 or x >= 20) else 0)  # Assuming night is 8PM to 6AM

# Keeping all features from the original train.csv
original_features = data_csv.columns.tolist()

weather_features = ['t', 'u', 'ff', 'rrN']  # temperature, humidity, wind speed, precipitation

# Discarding only the irrelevant features from external_data
irrelevant_external_features = [col for col in merged_data.columns if col not in original_features and col not in weather_features]
merged_data.drop(irrelevant_external_features, axis=1, inplace=True)

merged_data['hour'] = merged_data['date'].dt.hour
merged_data['day_of_week'] = merged_data['date'].dt.dayofweek
merged_data['month'] = merged_data['date'].dt.month
merged_data['is_weekend'] = merged_data['day_of_week'].isin([5, 6]).astype(int)  # 0 for weekdays, 1 for weekends
merged_data['is_night'] = merged_data['hour'].apply(lambda x: 1 if (x < 6 or x >= 20) else 0)  # Assuming night is 8PM to 6AM

In [278]:
def assign_time_of_day(hour):
    if 6 <= hour < 12:
        return "morning"
    elif 12 <= hour < 16:
        return "afternoon"
    elif 16 <= hour < 20:
        return "evening"
    else:
        return "night"

merged_data['time_of_day'] = merged_data['hour'].apply(assign_time_of_day)

# Encoding categorical variables (site_name, counter_name, time_of_day) using one-hot encoding
categorical_columns = ['site_name', 'counter_name', 'time_of_day']
merged_data = pd.get_dummies(merged_data, columns=categorical_columns)

# Normalizing/Standardizing numerical features
# Here, we'll use Min-Max Scaling as an example. Depending on the model, other methods like Standard Scaling could be used.
from sklearn.preprocessing import MinMaxScaler

numerical_columns = merged_data.select_dtypes(include=['int64', 'float64']).columns
numerical_columns = numerical_columns.drop('bike_count')  # Exclude target variable
numerical_columns = numerical_columns.drop('log_bike_count')  # Exclude target variable

scaler = MinMaxScaler()
merged_data[numerical_columns] = scaler.fit_transform(merged_data[numerical_columns])

# Displaying the first few rows of the prepared dataset
merged_data.head()


,counter_id,site_id,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,log_bike_count,ff,...,counter_name_Totem 85 quai d'Austerlitz NO-SE,counter_name_Totem 85 quai d'Austerlitz SE-NO,counter_name_Totem Cours la Reine E-O,counter_name_Totem Cours la Reine O-E,counter_name_Voie Georges Pompidou NE-SO,counter_name_Voie Georges Pompidou SO-NE,time_of_day_afternoon,time_of_day_evening,time_of_day_morning,time_of_day_night
0,100056332-104056332,0.000246,0.0,2020-09-01 01:00:00,2019-12-11 00:00:00,Y2H19070378,0.185435,0.765578,0.000000,0.125984,...,False,False,False,False,False,False,False,False,False,True
1,100047547-104047547,0.000202,4.0,2020-09-01 01:00:00,2018-11-28 00:00:00,Y2H18086323,0.000000,0.260692,1.609438,0.125984,...,False,False,False,False,False,False,False,False,False,True
2,100047547-103047547,0.000202,2.0,2020-09-01 01:00:00,2018-11-28 00:00:00,Y2H18086323,0.000000,0.260692,1.098612,0.125984,...,False,False,False,False,False,False,False,False,False,True
3,100057380-103057380,0.000252,0.0,2020-09-01 01:00:00,2020-02-11 00:00:00,YTH19111509,0.585373,0.339780,0.000000,0.125984,...,False,False,False,True,False,False,False,False,False,True
4,100047548-103047548,0.000202,2.0,2020-09-01 01:00:00,2018-11-28 00:00:00,Y2H18086324,0.995257,0.827615,1.098612,0.125984,...,False,False,False,False,False,False,False,False,False,True


In [279]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Preparing the data for modeling
X = merged_data.drop(['bike_count', 'log_bike_count', 'date', 'counter_id', 'counter_installation_date', 'counter_technical_id'], axis=1)
y = merged_data['log_bike_count']


In [282]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [284]:
# Model selection and training - using Random Forest for demonstration
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [285]:
# Predicting on the test set
y_pred = model.predict(X_test)


In [286]:
# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mse, r2

(0.15883643627462787, 0.94348231375068)

In [287]:
from sklearn.ensemble import GradientBoostingRegressor

# Model selection and training - using Gradient Boosting for better memory efficiency
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

# Predicting on the test set
y_pred_gb = gb_model.predict(X_test)

# Evaluating the model
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

mse_gb, r2_gb

(0.5752982160243496, 0.7952955578980623)

In [188]:
test_data = pd.read_parquet("final_test.parquet")

test_data['date'] = pd.to_datetime(test_data['date'])

new_data = test_data.reset_index()

# Preprocess the test data similar to the training data
# Including merging with external data, feature engineering, and encoding
test_data_merged = pd.merge_asof(new_data.sort_values('date'), 
                                 external_data_interpolated.sort_values('date'),
                                 on='date', 
                                 direction='nearest')

irrelevant_external_features = [col for col in test_data_merged.columns if col not in original_features and col not in weather_features and col != 'index']
test_data_merged.drop(irrelevant_external_features, axis=1, inplace=True)

# Adding date-related features to the test data
test_data_merged['hour'] = test_data_merged['date'].dt.hour
test_data_merged['day_of_week'] = test_data_merged['date'].dt.dayofweek
test_data_merged['month'] = test_data_merged['date'].dt.month
test_data_merged['is_weekend'] = test_data_merged['day_of_week'].isin([5, 6]).astype(int)
test_data_merged['is_night'] = test_data_merged['hour'].apply(lambda x: 1 if (x < 6 or x >= 20) else 0)
test_data_merged['time_of_day'] = test_data_merged['hour'].apply(assign_time_of_day)

In [189]:
test_data_merged['time_of_day'] = test_data_merged['hour'].apply(assign_time_of_day)

# Encoding categorical variables (site_name, counter_name, time_of_day) using one-hot encoding
categorical_columns = ['site_name', 'counter_name', 'time_of_day']
test_data_merged = pd.get_dummies(test_data_merged, columns=categorical_columns)


In [191]:
test_data_merged.sort_values(by = 'index', inplace=True)

In [194]:
test_data_merged.drop(columns='index', inplace=True)


In [197]:
# Normalizing/Standardizing numerical features
# Here, we'll use Min-Max Scaling as an example. Depending on the model, other methods like Standard Scaling could be used.
from sklearn.preprocessing import MinMaxScaler

numerical_columns = test_data_merged.select_dtypes(include=['int64', 'float64']).columns

scaler = MinMaxScaler()
test_data_merged[numerical_columns] = scaler.fit_transform(test_data_merged[numerical_columns])

# Displaying the first few rows of the prepared dataset
test_data_merged.head()


,counter_id,site_id,date,counter_installation_date,counter_technical_id,latitude,longitude,ff,t,u,...,counter_name_Totem 85 quai d'Austerlitz NO-SE,counter_name_Totem 85 quai d'Austerlitz SE-NO,counter_name_Totem Cours la Reine E-O,counter_name_Totem Cours la Reine O-E,counter_name_Voie Georges Pompidou NE-SO,counter_name_Voie Georges Pompidou SO-NE,time_of_day_afternoon,time_of_day_evening,time_of_day_morning,time_of_day_night
0,100007049-102007049,0.0,2021-09-10 01:00:00,2013-01-18,Y2H15027244,0.300918,0.762522,0.204082,0.659091,0.915254,...,False,False,False,False,False,False,False,False,False,True
719,100007049-102007049,0.0,2021-09-10 13:00:00,2013-01-18,Y2H15027244,0.300918,0.762522,0.183673,0.777273,0.830508,...,False,False,False,False,False,False,True,False,False,False
925,100007049-102007049,0.0,2021-09-10 17:00:00,2013-01-18,Y2H15027244,0.300918,0.762522,0.275510,0.759091,0.694915,...,False,False,False,False,False,False,False,True,False,False
1012,100007049-102007049,0.0,2021-09-10 19:00:00,2013-01-18,Y2H15027244,0.300918,0.762522,0.275510,0.759091,0.694915,...,False,False,False,False,False,False,False,True,False,False
1184,100007049-102007049,0.0,2021-09-10 22:00:00,2013-01-18,Y2H15027244,0.300918,0.762522,0.183673,0.650000,0.779661,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
scaler = MinMaxScaler()
merged_data[numerical_columns] = scaler.fit_transform(merged_data[numerical_columns])

# Displaying the first few rows of the prepared dataset
merged_data.head()


In [206]:
test_data_merged.drop(["counter_installation_date", "counter_technical_id"], axis=1, inplace=True)

In [210]:
test_data_merged.drop(["counter_id", "date"], axis=1, inplace=True)

In [211]:
test_data_merged.head()

,site_id,latitude,longitude,ff,t,u,hour,day_of_week,month,is_weekend,...,counter_name_Totem 85 quai d'Austerlitz NO-SE,counter_name_Totem 85 quai d'Austerlitz SE-NO,counter_name_Totem Cours la Reine E-O,counter_name_Totem Cours la Reine O-E,counter_name_Voie Georges Pompidou NE-SO,counter_name_Voie Georges Pompidou SO-NE,time_of_day_afternoon,time_of_day_evening,time_of_day_morning,time_of_day_night
0,0.0,0.300918,0.762522,0.204082,0.659091,0.915254,1,4,9,0,...,False,False,False,False,False,False,False,False,False,True
719,0.0,0.300918,0.762522,0.183673,0.777273,0.830508,13,4,9,0,...,False,False,False,False,False,False,True,False,False,False
925,0.0,0.300918,0.762522,0.275510,0.759091,0.694915,17,4,9,0,...,False,False,False,False,False,False,False,True,False,False
1012,0.0,0.300918,0.762522,0.275510,0.759091,0.694915,19,4,9,0,...,False,False,False,False,False,False,False,True,False,False
1184,0.0,0.300918,0.762522,0.183673,0.650000,0.779661,22,4,9,0,...,False,False,False,False,False,False,False,False,False,True


In [255]:
X_train.columns

Index(['site_id', 'latitude', 'longitude', 'ff', 't', 'u', 'hour',
       'day_of_week', 'month', 'is_weekend',
       ...
       'counter_name_Totem 85 quai d'Austerlitz NO-SE',
       'counter_name_Totem 85 quai d'Austerlitz SE-NO',
       'counter_name_Totem Cours la Reine E-O',
       'counter_name_Totem Cours la Reine O-E',
       'counter_name_Voie Georges Pompidou NE-SO',
       'counter_name_Voie Georges Pompidou SO-NE', 'time_of_day_afternoon',
       'time_of_day_evening', 'time_of_day_morning', 'time_of_day_night'],
      dtype='object', length=101)

In [257]:
test_data_merged.columns

Index(['site_id', 'latitude', 'longitude', 'ff', 't', 'u', 'hour',
       'day_of_week', 'month', 'is_weekend',
       ...
       'counter_name_Totem 85 quai d'Austerlitz NO-SE',
       'counter_name_Totem 85 quai d'Austerlitz SE-NO',
       'counter_name_Totem Cours la Reine E-O',
       'counter_name_Totem Cours la Reine O-E',
       'counter_name_Voie Georges Pompidou NE-SO',
       'counter_name_Voie Georges Pompidou SO-NE', 'time_of_day_afternoon',
       'time_of_day_evening', 'time_of_day_morning', 'time_of_day_night'],
      dtype='object', length=101)

In [295]:
# Model selection and training - using Random Forest for demonstration
model = RandomForestRegressor()
model.fit(X, y)


RandomForestRegressor()

In [289]:
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X, y)


GradientBoostingRegressor(random_state=42)

In [296]:
y_pred = model.predict(test_data_merged)

In [297]:
results = pd.DataFrame(
    dict(
        Id=np.arange(y_pred.shape[0]),
        log_bike_count=y_pred,
    )
)
results.to_csv("submission_GPT_RandomForest2.csv", index=False)

In [298]:
pip install lightgbm


  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/b3/f8/ee33e36194eb03a76eccf3adac3fba51f0e56fbd20609bb531659d48d3cb/lightgbm-4.1.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.3 MB 1.3 MB/s eta 0:00:01
   ----------- ---------------------------- 0.4/1.3 MB 2.9 MB/s eta 0:00:01
   ------------------ --------------------- 0.6/1.3 MB 3.5 MB/s eta 0:00:01
   -------------------------- ------------- 0.9/1.3 MB 4.2 MB/s eta 0:00:01
   ---------------------------------- ----- 1.1/1.3 MB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 4.4 MB/s eta 0:00:00


In [315]:
import lightgbm as lgb

# Create the LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Define the parameters for the model
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting': 'gbdt',
    'learning_rate': 0.01,
    'num_leaves': 31,
    'max_depth': -1,
    'min_child_samples': 20,
    'max_bin': 255,
    'subsample': 0.9,
    'subsample_freq': 1,
    'colsample_bytree': 1,
    'min_child_weight': 0.001,
    'subsample_for_bin': 200000,
    'min_split_gain': 0,
    'reg_alpha': 0,
    'reg_lambda': 0,
    'nthread': 4,
    'verbose': 0
}

# Train the model
num_round = 5000
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# Make predictions
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

# Evaluate the model
mse_lgbm = mean_squared_error(y_test, y_pred)
r2_lgbm = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse_lgbm}")
print(f"R-squared: {r2_lgbm}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


In [316]:
y_pred = bst.predict(test_data_merged, num_iteration=bst.best_iteration)

In [317]:
results = pd.DataFrame(
    dict(
        Id=np.arange(y_pred.shape[0]),
        log_bike_count=y_pred,
    )
)
results.to_csv("submission_LGB3.csv", index=False)

In [306]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

# Define the model
model = LGBMRegressor()

# Define the parameters for grid search
param_grid = {
    'num_leaves': [31, 50, 70],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 200],
    'max_depth': [-1, 10, 20],
    'min_child_samples': [20, 30, 50],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

# Define the grid search
grid = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

# Fit the grid search to the data
grid.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters found: ", grid.best_params_)
print("Best score found: ", np.sqrt(-grid.best_score_))

Fitting 3 folds for each of 972 candidates, totalling 2916 fits


KeyboardInterrupt: 

In [ ]:
# Use the best parameters to make predictions
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the predictions
mse_lgbm = mean_squared_error(y_test, y_pred)
r2_lgbm = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse_lgbm}")
print(f"R-squared: {r2_lgbm}")


In [ ]:
full_model = lgb.LGBMRegressor(grid.best_params_)
full_model.fit(X, y)


In [312]:
model = lgb.LGBMRegressor()
model.fit(X, y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005564 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 756
[LightGBM] [Info] Number of data points in the train set: 455163, number of used features: 101
[LightGBM] [Info] Start training from score 3.048589


LGBMRegressor()

In [313]:
y_pred = model.predict(test_data_merged)

results = pd.DataFrame(
    dict(
        Id=np.arange(y_pred.shape[0]),
        log_bike_count=y_pred,
    )
)
results.to_csv("submission_LGB2.csv", index=False)